In [1]:
% cd /home/mayu-ot/Documents/iparaphrasing-release/

/home/mayu-ot/Documents/iparaphrasing-release


In [2]:
import sys
sys.path.append('codes/script/detection')
from cluster_paraphrase import score_based_cluster, feat_based_cluster
sys.path.append('codes/script/eval')
from eval_cluster import eval_ari_score
from IPython.display import display_html

In [3]:
def read_best_param(report_file):
    with open(report_file) as f:
        for line in f:
            if line[:4] == 'Best':
                best_param = line.rstrip().split('  ')[-1]
    return float(best_param)

def compute_ari_score(method, eval_type):
    report_file = 'ari_data/bo_reports/%s/%s.bo.report' % (eval_type, method)
    best_val = read_best_param(report_file)
    split = 'test'
    up_file = 'data/entity/test/uniquePhrases'
    feat_file = 'data/entity/test/textFeats_%s.npy' % method
    saveto = 'output/cluster_res_%s.csv' % method
    
    gs_file = 'ari_data/gt_cluster-label_test_%s.csv' % eval_type
    res_df = feat_based_cluster(up_file,
                           gs_file, 
                           feat_file, 
                           clutter=best_val, 
                           damping=0.5,
                           affinity='cosine')
    score = eval_ari_score(gs_file, res_df)
    return score

def compute_ari_score_by_pair_score(method, eval_type):
    report_file = 'ari_data/bo_reports/%s/%s.bo.report' % (eval_type, method)
    best_val = read_best_param(report_file)
    split = 'test'
    up_file = 'data/entity/test/uniquePhrases'
    
    results = {}

    gs_file = 'ari_data/gt_cluster-label_test_%s.csv' % eval_type
    pair_score_file = 'models/%s/res_test.csv' % method
    affinity_dir = 'output/af/%s/test/%s/' % (method, eval_type)
    res_df = score_based_cluster(gs_file,pair_score_file,affinity_dir
                ,clutter=best_val,damping=0.5)
    score = eval_ari_score(gs_file, res_df)
    return score

In [4]:
import os

methods = [x for x in os.listdir('./models') if x[0] != '.']
res = {}
for method_name in methods:
    if method_name in ['fv+cca', 'fv+pca']:
        scores = {compute_ari_score(method_name, eval_type) for eval_type in ['all', 'single', 'multiple']}
    else:
        scores = {compute_ari_score_by_pair_score(method_name, eval_type) for eval_type in ['all', 'single', 'multiple']}
        
    res[method_name] = scores

In [5]:
head = '''
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;border:none;border-color:#ccc;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#fff;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;border-color:#ccc;color:#333;background-color:#f0f0f0;}
.tg .tg-88nc{font-weight:bold;border-color:inherit;text-align:center}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-uys7{border-color:inherit;text-align:center}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-uys7"></th>
    <th class="tg-88nc" colspan="3">ARI</th>
  </tr>
  <tr>
    <td class="tg-c3ow"></td>
    <td class="tg-7btt">all</td>
    <td class="tg-7btt">single</td>
    <td class="tg-7btt">multi</td>
  </tr>
'''

row = '''
  <tr>
    <td class="tg-88nc">%s</td>
    <td class="tg-uys7">%.2f<br></td>
    <td class="tg-uys7">%.2f</td>
    <td class="tg-uys7">%.2f</td>
  </tr>
'''

bottom = '''
</table>
'''

methods = {
    'PL': 'pl',
    'TP': 'tp',
    'WEA': 'wea',
    'FV': 'fv+pca',
    'FV+CCA': 'fv+cca',
    'SNN(WEA)': 'avr-None',
    'SNN+image (WEA)': 'avr-vgg',
    'Ensemble (WEA)': 'Ens_avr',
    'SNN(FV)': 'fv+pca-None',
    'SNN+image (FV)': 'fv+pca-vgg',
    'Ensemble (FV)': 'Ens_fv+pca',
    'SNN (FV+CCA)': 'fv+cca-None',
    'SNN+image (FV+CCA)': 'fv+cca-vgg',
    'ensemble (FV+CCA)': 'Ens_fv+cca'
    
}

In [6]:
html = head

# for method, scores in res.items():
for k, v in methods.items():
    all_s, single_s, mult_s = res[v]
    html += row % (k, 100*all_s, 100*single_s, 100*mult_s)

html += bottom

display_html(html, raw=True)

ARI 
 
 
 
 all 
 single 
 multi 
 

 
 PL 
 43.23 
 45.92 
 46.35 
 

 
 TP 
 37.61 
 50.32 
 36.79 
 

 
 WEA 
 49.82 
 47.16 
 49.58 
 

 
 FV 
 39.85 
 43.55 
 41.26 
 

 
 FV+CCA 
 54.91 
 51.01 
 49.30 
 

 
 SNN(WEA) 
 60.23 
 55.06 
 53.26 
 

 
 SNN+image (WEA) 
 60.55 
 55.42 
 55.82 
 

 
 Ensemble (WEA) 
 60.65 
 54.92 
 54.56 
 

 
 SNN(FV) 
 48.13 
 45.97 
 47.22 
 

 
 SNN+image (FV) 
 47.90 
 47.39 
 48.31 
 

 
 Ensemble (FV) 
 49.82 
 48.16 
 48.34 
 

 
 SNN (FV+CCA) 
 60.56 
 56.35 
 54.06 
 

 
 SNN+image (FV+CCA) 
 61.17 
 54.86 
 54.14 
 

 
 ensemble (FV+CCA) 
 62.35 
 54.98 
 54.84